In [1]:
# Add src folder to Python path so db_config can be imported
import sys
import os

sys.path.append(os.path.abspath("../src"))

In [2]:
# Used to execute SQL queries and show results
from sqlalchemy import text
from db_config import engine
import pandas as pd

# Table record counts


In [ ]:

def record_count(table):
    with engine.connect() as conn:
        result = conn.execute(
            text(f"SELECT COUNT(*) FROM {table}")
        ).scalar()
    return  result

#### we have written  a dynamic function which will take table name as a parameter and return count of records

In [13]:
record_count('sleepday_merged')

413

In [20]:


def get_all_tables():
    with engine.connect() as conn:
        result = conn.execute(
            text("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = :db_name
                  AND table_type = 'BASE TABLE'
            """),
            {"db_name": "fitbit_db"}   # 👈 pass DB name here
        )
        return [row[0] for row in result]


####  get_all_tables function  will return all  existing tables in the fitbit_db database

In [22]:
tables=get_all_tables()

>* we have stored all tables in a variable tables 
>* we used for loop to through tables and printed count of records

In [28]:
dict={}
for table in tables:
    res=record_count(table)
    dict[table]=res
    print(f'{table} :  {res}')
    print('-'*40)



dailyactivity_merged :  940
----------------------------------------
dailycalories_merged :  940
----------------------------------------
dailyintensities_merged :  940
----------------------------------------
dailysteps_merged :  940
----------------------------------------
heartrate_seconds_merged :  2483658
----------------------------------------
hourlycalories_merged :  22099
----------------------------------------
hourlyintensities_merged :  22099
----------------------------------------
hourlysteps_merged :  22099
----------------------------------------
minutecaloriesnarrow_merged :  1325580
----------------------------------------
minutecalorieswide_merged :  21645
----------------------------------------
minuteintensitiesnarrow_merged :  1325580
----------------------------------------
minuteintensitieswide_merged :  21645
----------------------------------------
minutemetsnarrow_merged :  1325580
----------------------------------------
minutesleep_merged :  188521
--------

# Unique user counts

In [35]:
with engine.connect() as conn:
        result = conn.execute(
            text(f"SELECT COUNT(DISTINCT ID) FROM dailyactivity_merged")
        ).scalar()
print(f'Unique Users:{result}')


Unique Users:33


# Date coverage per table

In [38]:
with engine.connect() as conn:
        result = conn.execute(
            text(f"SELECT MIN(ACTIVITYDATE),MAX(ACTIVITYDATE)  FROM dailyactivity_merged")
        ).fetchall()
result

[(datetime.date(2016, 4, 12), datetime.date(2016, 5, 12))]

#### we have found that the data is collected from 2016/04/12 to 2016/05/12(31 days)

# Distributions of steps, calories, sleep

In [56]:
with engine.connect() as conn:
        result = conn.execute(
            text(f"SELECT id, avg(Calories) as  'mean colories' FROM dailyactivity_merged group by id")
        )
df = pd.DataFrame(result.fetchall(), columns=result.keys())
df

,id,mean colories
0,1503960366,1816.4194
1,1624580081,1483.3548
2,1644430081,2811.3000
3,1844505072,1573.4839
4,1927972279,2172.8065
5,2022484408,2509.9677
6,2026352035,1540.6452
7,2320127002,1724.1613
8,2347167796,2043.4444
9,2873212765,1916.9677


In [52]:
with engine.connect() as conn:
        result = conn.execute(
            text(f"SELECT id,avg(TotalSteps) as mean_steps  FROM dailyactivity_merged group by id")
        )

df = pd.DataFrame(result.fetchall(), columns=result.keys())
df

,id,mean_steps
0,1503960366,12116.7419
1,1624580081,5743.9032
2,1644430081,7282.9667
3,1844505072,2580.0645
4,1927972279,916.1290
5,2022484408,11370.6452
6,2026352035,5566.8710
7,2320127002,4716.8710
8,2347167796,9519.6667
9,2873212765,7555.7742


In [57]:
tables

['dailyactivity_merged',
 'dailycalories_merged',
 'dailyintensities_merged',
 'dailysteps_merged',
 'heartrate_seconds_merged',
 'hourlycalories_merged',
 'hourlyintensities_merged',
 'hourlysteps_merged',
 'minutecaloriesnarrow_merged',
 'minutecalorieswide_merged',
 'minuteintensitiesnarrow_merged',
 'minuteintensitieswide_merged',
 'minutemetsnarrow_merged',
 'minutesleep_merged',
 'minutestepsnarrow_merged',
 'minutestepswide_merged',
 'sleepday_merged',
 'weightloginfo_merged']

In [59]:
with engine.connect() as conn:
        result = conn.execute(
            text(f"SELECT id,avg(TotalTimeInBed) as mean_TotalTimeInBed FROM sleepday_merged group by id")
        )

df = pd.DataFrame(result.fetchall(), columns=result.keys())
df

,id,mean_TotalTimeInBed
0,1503960366,383.2000
1,1644430081,346.0000
2,1844505072,961.0000
3,1927972279,437.8000
4,2026352035,537.6429
5,2320127002,69.0000
6,2347167796,491.3333
7,3977333714,461.1429
8,4020332650,379.7500
9,4319703577,501.9615
